In [47]:
# importing modules
import numpy as np

In [48]:
# tensorflow modules and utils
import tensorflow as tf
import tf_keras as keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds 

In [49]:
# Info and Spliting of Data
splits ,  data_info = tfds.load(name = "imdb_reviews" , 
              split = (["train[:60%]" , "train[60%:]" , "test"]) , 
             as_supervised = True,
             with_info = True
            )

train_data , validation_data , test_data = splits

data_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_e

In [50]:
# training data , validation data , testing data 
train_data , validation_data , test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [51]:
# example batch of data 
train_batch_data , train_batch_label  = next(iter(train_data.batch(10)))

train_batch_data

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [52]:
print(train_batch_label)

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [53]:
# Model Building
embedding = "https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1"
hub_layer = hub.KerasLayer(embedding , input_shape = [] , dtype=tf.string , trainable = True)
hub_layer

In [54]:
hub_layer(train_batch_data[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [57]:
model = keras.Sequential([
    hub_layer,
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_6 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_4 (Dense)             (None, 16)                336       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
model.compile(optimizer = "adam" , loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [60]:
history = model.fit(train_data.shuffle(10000).batch(100),
                    epochs=25,
                    validation_data=validation_data.batch(100),
                    verbose=1)

Epoch 1/25
150/150 [==============================] - 3s 16ms/step - loss: 0.2812 - accuracy: 0.8805 - val_loss: 0.3300 - val_accuracy: 0.8575
Epoch 2/25
150/150 [==============================] - 3s 16ms/step - loss: 0.2233 - accuracy: 0.9101 - val_loss: 0.3139 - val_accuracy: 0.8622
Epoch 3/25
150/150 [==============================] - 3s 16ms/step - loss: 0.1878 - accuracy: 0.9275 - val_loss: 0.3100 - val_accuracy: 0.8623
Epoch 4/25
150/150 [==============================] - 3s 16ms/step - loss: 0.1604 - accuracy: 0.9399 - val_loss: 0.3127 - val_accuracy: 0.8621
Epoch 5/25
150/150 [==============================] - 3s 16ms/step - loss: 0.1379 - accuracy: 0.9510 - val_loss: 0.3170 - val_accuracy: 0.8659
Epoch 6/25
150/150 [==============================] - 3s 16ms/step - loss: 0.1191 - accuracy: 0.9595 - val_loss: 0.3334 - val_accuracy: 0.8729
Epoch 7/25
150/150 [==============================] - 3s 16ms/step - loss: 0.1032 - accuracy: 0.9669 - val_loss: 0.3370 - val_accuracy: 0.8686

In [61]:
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

250/250 - 2s - loss: 0.7760 - accuracy: 0.8360 - 2s/epoch - 9ms/step
loss: 0.776
accuracy: 0.836
